# Базы данных для аналитиков. Основы ETL
## Домашнее задание 1. Аналитика в бизнес-задачах.

В это домашней работе я буду использоавать связку `sqlite3` + `jupyter-notebook`

Подгружаем модуль `SQL`

In [1]:
%load_ext sql

#### 1. Залить в свою БД данные по продажам
_Идея закинуть csv-файлы в `sqlite` оказалась не просто выполнимая.
Во-первых, формат даты не подходит для `sqlite` - пришлось использовать регулярные выражения чтобы конвертировать дату:_

In [9]:
"""

# !/bin/bash
# Конвертация даты
sed -E -i  's,([0-9]{2})\.([0-9]{2})\.([0-9]{4}),\3-\2-\1,g' orders_20190822.csv

# Создание базы sqlite3 и импортирование csv
echo '
.mode csv  
.separator ";"
.import orders_20190822.csv orders
.import basket_20190922.csv basket
' | sqlite3 h1.db

""";

In [2]:
%sql sqlite:///h1.db

'Connected: @h1.db'

#### 2. Проанализировать, какой период данных выгружен

In [3]:
%%sql 
SELECT 
    min(o_date) as 'Начало временного периода', 
    max(o_date) as 'Конец временного периода'
FROM orders;

 * sqlite:///h1.db
Done.


Начало временного периода,Конец временного периода
2016-01-01,2017-12-31


#### 3. Посчитать кол-во строк, кол-во заказов и кол-во уникальных пользователей, кот совершали заказы.
_Не совсем понятно по поводу количества строк, потому опустим этот признак_

In [4]:
%%sql
SELECT
    count(o.id_o) as 'Количество заказов',
    count(DISTINCT o.user_id) as 'Количество уникальных пользователей'
FROM orders o;

 * sqlite:///h1.db
Done.


Количество заказов,Количество уникальных пользователей
2002804,1015119


####  4. По годам посчитать средний чек, среднее кол-во заказов на пользователя, сделать вывод , как изменялись это показатели Год от года.

In [5]:
%%sql
SELECT 
    AVG("Средний чек") as 'Средний чек', 
    AVG("Количество заказов") as 'Среднее количество заказов на пользователя', 
    "Год"
FROM (
    SELECT 
        avg(o.price) as 'Средний чек', 
        count(o.id_o) as 'Количество заказов', 
        o.user_id as 'Пользователь', 
        strftime('%Y', o_date) as 'Год'
    FROM orders o
    GROUP BY o.user_id
)
GROUP BY "Год"


 * sqlite:///h1.db
Done.


Средний чек,Среднее количество заказов на пользователя,Год
2184.3714636013306,2.67649160173627,2016
2427.0430382262994,1.423650107801911,2017


#### 5. Найти кол-во пользователей, которые покупали в одном году и перестали покупать в следующем.

In [6]:
%%sql
SELECT
    count(DISTINCT user_id) as 'Количество пользователей, переставших покупать в новом году'
FROM orders
WHERE 
    strftime('%Y', o_date) = '2016'
    and strftime('%Y', o_date) <> '2017'
;

 * sqlite:///h1.db
Done.


"Количество пользователей, переставших покупать в новом году"
445092


#### 6. Найти IDсамого активного по кол-ву покупок пользователя. 

In [7]:
%%sql
SELECT user_id, "Количесвто покупок"
FROM (
    SELECT
        user_id, 
        count(id_o) as 'Количесвто покупок'
    FROM orders
    GROUP BY user_id
)
ORDER BY "Количесвто покупок" DESC
LIMIT 1;

 * sqlite:///h1.db
Done.


user_id,Количесвто покупок
765861,3183
